## 1. Configuration Initiale

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import Perceptron as SklearnPerceptron
import plotly.express as px

## 2. Chargement et Exploration des Données

In [ ]:
df = pd.read_csv('Iris.csv')
print("Statistiques descriptives :\n", df.describe())
print("\nDistribution des classes :\n", df['Species'].value_counts())

# Visualisation interactive 3D
fig = px.scatter_3d(df, x='SepalLengthCm', y='SepalWidthCm', z='PetalLengthCm', 
                    color='Species', title="Distribution 3D des Espèces")
fig.show()

 ## 3. Prétraitement des Données

In [ ]:
# Filtrage des classes et sélection de features
df = df[df['Species'].isin(['Iris-setosa', 'Iris-versicolor'])]
X = df[['SepalLengthCm', 'PetalWidthCm']].values  # Choix stratégique de features
y = df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1}).values

# Normalisation
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

 ## 4. Implémentation Avancée du Perceptron

In [ ]:
class MyPerceptron:
    def __init__(self, learning_rate=0.1, n_iters=100, early_stopping=True):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        self.early_stopping = early_stopping
        self.errors_history = []
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        for epoch in range(self.n_iters):
            total_error = 0
            for xi, yi in zip(X, y):
                prediction = self.activation(xi)
                error = yi - prediction
                update = self.lr * error
                self.weights += update * xi
                self.bias += update
                total_error += int(error != 0)
            
            self.errors_history.append(total_error)
            if self.early_stopping and total_error == 0:
                print(f"Arrêt précoce à l'epoch {epoch}")
                break
                
    def activation(self, x):
        return 1 if (np.dot(x, self.weights) + self.bias) >= 0 else 0
    
    def predict(self, X):
        return np.array([self.activation(x) for x in X])

## 5. Entraînement et Visualisation Dynamique

In [ ]:
perceptron = MyPerceptron(learning_rate=0.15, n_iters=50)
perceptron.fit(X_train, y_train)

# Visualisation de l'évolution des erreurs
plt.plot(perceptron.errors_history, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Erreurs de Classification')
plt.title('Convergence du Perceptron')
plt.grid(True)
plt.show()

## 6. Visualisation Avancée des Frontières

In [ ]:
def plot_decision_boundary_3d(model, X, y):
    # Création d'une grille 3D
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 50),
                         np.linspace(y_min, y_max, 50))
    
    # Prédictions sur la grille
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Visualisation 3D interactive
    fig = px.scatter_3d(x=X[:,0], y=X[:,1], z=y, 
                        color=y.astype(str), 
                        labels={'color': 'Classe'},
                        title="Frontière de Décision 3D")
    fig.update_traces(marker=dict(size=5))
    fig.add_surface(x=xx, y=yy, z=Z, opacity=0.5, colorscale='Blues')
    fig.show()

plot_decision_boundary_3d(perceptron, X_train, y_train)

 ## 7. Évaluation Comparative

In [ ]:
# Évaluation du modèle personnalisé
y_pred_custom = perceptron.predict(X_test)
print("Performance du Perceptron Personnalisé :")
print(f"Accuracy: {accuracy_score(y_test, y_pred_custom):.2f}")

# Comparaison avec Scikit-Learn
sk_perceptron = SklearnPerceptron(alpha=0.15, max_iter=50)
sk_perceptron.fit(X_train, y_train)
y_pred_sk = sk_perceptron.predict(X_test)
print("\nPerformance du Perceptron de Scikit-Learn :")
print(f"Accuracy: {accuracy_score(y_test, y_pred_sk):.2f}")

# Matrice de confusion interactive
fig = px.imshow(confusion_matrix(y_test, y_pred_custom),
                labels=dict(x="Prédit", y="Réel", color="Count"),
                x=['0', '1'], y=['0', '1'],
                title="Matrice de Confusion")
fig.show()


## 9. Test Interactif avec Boucle

In [ ]:

def test_model_interactively(model, scaler):
    print("Entrez les valeurs pour tester le modèle :")
    sepal_length = float(input("Longueur du sépale (en cm) : "))
    petal_width = float(input("Largeur du pétale (en cm) : "))
    
    input_data = np.array([[sepal_length, petal_width]])
    
    input_data_normalized = scaler.transform(input_data)
    
    prediction = model.predict(input_data_normalized)
    predicted_class = "Setosa" if prediction[0] == 0 else "Versicolor"
    
    print(f"\nRésultat de la prédiction : {predicted_class}")

test_model_interactively(perceptron, scaler)